#### Project Description

Solar energy plays a crucial role in reducing greenhouse gas emissions and meeting rising energy demands. Information on building roof types and geometry is essential for assessing solar potential, improving energy efficiency, and supporting energy transition policies. Roof material and shape data also can inform studies on thermal efficiency, roof durability, disaster risk, heritage conservation, urban heat, and more.

This study aims to develop a ready-to-use, AI-ready open dataset containing building roof types and geometry, along with a scalable pipeline for roof material classification. The dataset is created using OpenStreetMap (OSM) roof material labels and high-resolution satellite imagery obtained from [OpenAerialMap](https://map.openaerialmap.org/#/7.164459228515626,50.84562199133437,12/square/12020303222/617970f8800b10000509eda7?_k=h73n1n), originally from the Maxar [Open Data Program](https://www.maxar.com/open-data), leveraging OSM building outlines to avoid computationally expensive segmentation processes. This notebook includes the code to obtain and preprocess this data, which can be used to transform any geospatial dataset into the YOLO dataset format, in addition to training a model.

To demonstrate the dataset’s applicability, a convolutional neural network (CNN) model, specifically, Ultralytics' YOLOv11, is used to classify roofs into categories such as roof tiles, tar paper, and metal, followed by an evaluation of the model’s performance.

#### Step 0: Ensure dependencies

In [ ]:
%pip install pyyaml numpy geopandas matplotlib rasterio shapely opencv-python overpy ultralytics

In [3]:
import os
import yaml
import numpy as np
import geopandas as gpd
import matplotlib as plt
import rasterio
from rasterio.windows import Window
from shapely.geometry import box, Polygon
import cv2
import overpy
from ultralytics import YOLO

#### Step 1: Data Collection and Preprocessing

First, we query all the roofs that have a roof:material OpenStreetMap(OSM) tag in the study area. We will be using the Overpass API, which allows us to fetch data from OSM. More information about the Overpass API query language and other examples can be found here: https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example.

We recommend using taginfo's regional databases to get a better understanding of where OpenStreetMap Data you are looking for exists https://taginfo.geofabrik.de/. 

In [4]:
import overpy

def query_osm(bbox: tuple[float, float, float, float]) -> list[dict[str, int|str|list[tuple[float, float]]]]:
    """
    Queries OpenStreetMap (OSM) using Overpass API for buildings within a given bounding box
    that have roof material data.

    Args:
        bbox (tuple[float, float, float, float]): Bounding box coordinates in the format 
            (min_latitude, min_longitude, max_latitude, max_longitude).

    Returns:
        list[dict[str, int | str | list[tuple[float, float]]]]: A list of dictionaries where each dictionary represents a building.
            Each dictionary contains:
            - "id" (int): OSM way ID.
            - "roof_material" (str): Roof material type or "unknown" if not specified.
            - "geometry" (list[tuple[float, float]]): List of (longitude, latitude) tuples 
              representing the building's outline.
    """
    api = overpy.Overpass()
    query = f"""
[out:json][timeout:180];
(
way["building"]["roof:material"]{bbox};
);
out body;
>;
out skel qt;
"""

    response = api.query(query)

    bldgs = []
    for way in response.ways:
        building = {
        "id": way.id,
        "roof_material": way.tags.get("roof:material", "unknown"),
        "geometry": [(float(node.lon), float(node.lat)) for node in way.nodes]
        #shapely expects longitude, latitude, in addition to GeoJSONs, even though WGS894 uses latitude, longitude
    }
        bldgs.append(building)
    return bldgs

# change the bounding box coordinates to get data from your desired study area (min latitude, min longitude, max latitude, max longitude)
bbox = (50.47044, 7.04552, 50.917936, 7.271421)
# Cologne: 49.97138, 6.69497, 51.02720390596486, 6.907890818310556
# Bonn: 50.47044, 7.04552, 50.917936, 7.271421
# Meckenheim: 50.13682, 6.90614, 50.5464, 7.0739
buildings_material = query_osm(bbox)

print(f"total buildings queried: {len(buildings_material)}")
print(f"first 4 buildings: {buildings_material[0:3]}")

total buildings queried: 19375
first 4 buildings: [{'id': 22376701, 'roof_material': 'roof_tiles', 'geometry': [(7.1748783, 50.8147237), (7.1749393, 50.8144985), (7.1748001, 50.8144834), (7.1747545, 50.8146516), (7.1747391, 50.8147086), (7.1748783, 50.8147237)]}, {'id': 22679422, 'roof_material': 'tar_paper', 'geometry': [(7.1582354, 50.8127516), (7.1582094, 50.8127861), (7.1577939, 50.8126591), (7.157824, 50.8126201), (7.157807, 50.8126066), (7.1578146, 50.8125957), (7.1578082, 50.8125928), (7.157812, 50.8125864), (7.1577886, 50.8125731), (7.1577803, 50.8125757), (7.1577763, 50.812571), (7.1577852, 50.812568), (7.1577767, 50.8125463), (7.1577662, 50.8125459), (7.1577669, 50.81254), (7.1577767, 50.8125404), (7.1577924, 50.8125227), (7.157785, 50.812518), (7.1577923, 50.8125136), (7.1577991, 50.8125181), (7.1578275, 50.8125085), (7.1578268, 50.8125011), (7.157838, 50.8125002), (7.1578383, 50.8125072), (7.1578711, 50.8125086), (7.157875, 50.8125035), (7.1578838, 50.8125041), (7.1578926, 

In [5]:
def create_building_material_gdf(buildings):
    """
    Creates a GeoDataFrame from a list of building data, standardizes roof material labels,
    filters out rare categories, and saves the data as GeoJSON files.

    Args:
        buildings (list[dict[str, int | str | list[tuple[float, float]]]]): A list of dictionaries representing buildings,
            each containing:
            - "id" (int): The OSM way ID.
            - "roof_material" (str): The type of roof material.
            - "geometry" (list[tuple[float, float]]): List of (longitude, latitude) tuples.

    Returns:
        gpd.GeoDataFrame: Filtered GeoDataFrame containing buildings with more common roof materials.
    """
    geometries = [Polygon(building["geometry"]) for building in buildings]
    roof_materials = [building["roof_material"] for building in buildings]

    gdf = gpd.GeoDataFrame({"roof_material": roof_materials}, geometry=geometries)
    gdf.set_crs(epsg=4326, inplace=True)

    # filter roof categories to ensure 
    category_counts = gdf["roof_material"].value_counts()
    keep_materials = category_counts[category_counts > 10].index.tolist()
    gdf_filtered = gdf[gdf['roof_material'].isin(keep_materials)]

    output_file = "bonn_buildings.geojson"
    gdf.to_file(output_file, driver="GeoJSON")

    output_file_filtered = "bonn_buildings_filtered.geojson"
    gdf_filtered.to_file(output_file_filtered, driver="GeoJSON")
    return gdf_filtered

gdf_filtered = create_building_material_gdf(buildings_material)
print(gdf_filtered['roof_material'].value_counts())


roof_material
roof_tiles    13438
tar_paper      5520
metal           109
wood             70
eternit          49
gravel           45
glass            33
grass            32
concrete         28
slate            21
sheet            14
Name: count, dtype: int64


After saving the queried OpenStreetMap data to a GeoJSON, we manually align the data and fix incorrect labels in QGIS. For the purpose of the sample dataset demonstration, we will just use the GeoJSON that was just generated.

#### Step 2: Create AI-Ready Dataset
In this step, we present a general Python module to create a training dataset in the [Ultralytics YOLO instance segementation dataset format](https://docs.ultralytics.com/datasets/segment/) from geospatial data, saved to the notebook's directory. 

This dataset takes in a raster imagery file, then 

In [12]:
class RoofMaterialDatasetConverter:
    def __init__(self, 
                 geojson_path: str, 
                 raster_path: str, 
                 output_dir: str, 
                 patch_size: int = 384, 
                 overlap: int = 128,
                 patch_min_buildings: int = 3) -> None:

        self.gdf = gpd.read_file(geojson_path)
        self.raster = rasterio.open(raster_path)
        
        self.output_dir = output_dir
        
        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)
        
        self.patch_size = patch_size
        self.overlap = overlap
        self.patch_min_buildings = patch_min_buildings

        self.class_mapping = {
            'gabled': 0,
            'flat': 1,
            'hipped': 2,
            'skillion': 3,
            'gambrel': 4,
            'half-hipped': 5,
            'pyramidal': 6,
            'mansard': 7
        }
        # self.class_mapping = {
        #     'roof_tiles': 0,
        #     'tar_paper': 1,
        #     'metal': 2,
        #     'concrete': 3,
        #     'tile': 4,
        #     'gravel': 5,
        #     'glass': 6
        # }

    def generate_patches(self, 
                     train_ratio: float = 0.7, 
                     val_ratio: float = 0.15, 
                     random_state: int = 42) -> dict[str, list[tuple[str, str]]]:
        """
        Generate image patches with corresponding roof material labels.
        
        Returns:
            list of tuples: [(image_path, label_path), ...]
        """
        rng = np.random.default_rng(random_state)

        # collect all potential patches
        all_patches: list[tuple[str, str]] = []
        
        height, width = self.raster.height, self.raster.width
        for y in range(0, height, self.patch_size - self.overlap):
            for x in range(0, width, self.patch_size - self.overlap):
                window = Window(x, y, self.patch_size, self.patch_size)
                
                # read patch from raster
                try:
                    patch = self.raster.read(window=window)
                    patch = np.transpose(patch[:3], (1, 2, 0))  # transpose to HWC
                except Exception as e:
                    print(f"Skipping patch at ({x},{y}) due to error: {e}")
                    continue
                
                # check if patch contains any roof material labels
                patch_bounds = box(
                    self.raster.xy(y, x)[0], 
                    self.raster.xy(y, x)[1],
                    self.raster.xy(y + self.patch_size, x + self.patch_size)[0],
                    self.raster.xy(y + self.patch_size, x + self.patch_size)[1]
                )
                
                # filter roof polygons within this patch
                patch_roofs = self.gdf[self.gdf.intersects(patch_bounds)]
                
                if len(patch_roofs) >= self.patch_min_buildings:
                    label_content = self._generate_labels(patch_roofs, patch_bounds)
                    if label_content:
                        all_patches.append((patch, (y, x), label_content))
        
        # randomly split patches
        n_patches = len(all_patches)
        train_size = int(n_patches * train_ratio)
        val_size = int(n_patches * val_ratio)
        
        rng.shuffle(all_patches)
        
        train_patches = all_patches[:train_size]
        val_patches = all_patches[train_size:train_size+val_size]
        test_patches = all_patches[train_size+val_size:]
        
        split_paths = {
            'train': [],
            'val': [],
            'test': []
        }
        
        for split, patches in [('train', train_patches), ('val', val_patches), ('test', test_patches)]:
            for patch, (y, x), label_content in patches:
                patch_filename = f'patch_{y}_{x}.png'
                
                img_path = os.path.join(self.output_dir, 'images', split, patch_filename)
                label_path = os.path.join(self.output_dir, 'labels', split, patch_filename.replace('.png', '.txt'))
                
                os.makedirs(os.path.dirname(img_path), exist_ok=True)
                os.makedirs(os.path.dirname(label_path), exist_ok=True)
                
                cv2.imwrite(img_path, cv2.cvtColor(patch, cv2.COLOR_RGB2BGR))
                
                with open(label_path, 'w') as f:
                    f.write('\n'.join(label_content))
                split_paths[split].append((img_path, label_path))
        
        return split_paths

    def _generate_labels(self, 
                         roof_polygons: gpd.GeoDataFrame,
                         patch_bounds: box) -> list[str]:
        """
        Generate YOLO segmentation labels for roof polygons in a patch.
        
        Parameters:
            roof_polygons : GeoDataFrame
                GeoDataFrame of roof polygons in the current patch
            patch_bounds : shapely.geometry.box
                Bounding box of the current patch
        
        Returns:
            list of label strings
        """
        labels: list[str] = []
        
        for _, roof in roof_polygons.iterrows():
            # only include specified roof material classes
            class_name = roof['roof_shape']
            if class_name not in self.class_mapping:
                continue
            class_index = self.class_mapping[class_name]
            
            # clip polygon to patch bounds
            clipped_roof = roof.geometry.intersection(patch_bounds)
            
            if not clipped_roof.is_empty:
                try:
                    coords = np.array(clipped_roof.exterior.coords)
                    normalized_coords = self._normalize_coordinates(coords, patch_bounds)
                    
                    if len(normalized_coords) >= 3:
                        # create label string: class_index x1 y1 x2 y2 ...
                        label = f"{class_index} " + " ".join(map(str, normalized_coords.flatten()))
                        labels.append(label)
                except Exception as e:
                    print(f"Error processing roof polygon: {e}")
        
        return labels
    
    def _normalize_coordinates(self, 
                               coords: np.ndarray, 
                               patch_bounds: box) -> np.ndarray:
        """
        Normalize polygon coordinates to 0-1 range within patch.
        
        Parameters:
            coords: numpy.ndarray
                Original polygon coordinates
            patch_bounds : shapely.geometry.box
                Bounding box of the current patch
        
        Returns:
            numpy.ndarray of normalized coordinates
        """
        x_min, y_min = patch_bounds.bounds[0], patch_bounds.bounds[1]
        x_max, y_max = patch_bounds.bounds[2], patch_bounds.bounds[3]
        normalized_coords = coords.copy()
        normalized_coords[:, 0] = (coords[:, 0] - x_min) / (x_max - x_min)
        #normalized_coords[:, 1] = (coords[:, 1] - y_min) / (y_max - y_min)
        normalized_coords[:, 1] = (y_max - coords[:, 1]) / (y_max - y_min)

        return normalized_coords
    
    def create_dataset_yaml(self) -> str:
        """
        Create YOLO dataset configuration YAML file.
        
        Returns:
            str: Path to the created YAML file
        """
        # reverse order for right YAML formatting
        class_names = {v: k for k, v in self.class_mapping.items()}
        
        yaml_content = {
            'path': self.output_dir,
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'names': class_names
        }
        
        yaml_path = os.path.join(self.output_dir, 'roof_materials.yaml')
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, default_flow_style=False)
        
        return yaml_path


In [ ]:
geojson_path = 'bonn_roof_shapes_aligned.geojson'
raster_path = '61796764800b10000509eda2.tif'
output_dir = 'datasets/bonn_dataset_shape'
# ran this for bonn_aligned (roof materials) small and bonn_roof_shape_aligned

converter = RoofMaterialDatasetConverter(
    geojson_path, 
    raster_path, 
    output_dir,
    patch_min_buildings=3
)

dataset_splits = converter.generate_patches(
    train_ratio=0.7,
    val_ratio=0.15,
    random_state=42
)

yaml_path = converter.create_dataset_yaml()

print(f"YOLO dataset created at {output_dir}")
print(f"Train patches: {len(dataset_splits['train'])}")
print(f"Validation patches: {len(dataset_splits['val'])}")
print(f"Test patches: {len(dataset_splits['test'])}")
print(f"Dataset YAML created at {yaml_path}")


#### Step 3: Model Training

Next, we train the pre-trained [Ultralytics YOLOv11](https://docs.ultralytics.com/models/yolo11/) instance segmentation model on our roof material dataset.

In [ ]:
model = YOLO('yolo11s-seg.pt')

results = model.train(
    data='datasets/bonn_dataset_shape/roof_materials.yaml',
    epochs=100,
    imgsz=320,
    batch=16,
    plots=True,
    device='0',
    pretrained=True,
    patience=20,
    save_period=10,
    name=f'run_5_shape'
)

New https://pypi.org/project/ultralytics/8.3.105 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.100 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40326MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=datasets/bonn_dataset_shape/roof_materials.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=320, save=True, save_period=10, cache=False, device=0, workers=8, project=None, name=run_5_shape, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

train: Scanning /home/ubuntu/roofmap_project/datasets/bonn_dataset_shape/labels/train.cache... 5556 images, 0 ba

train: WARNING ⚠️ /home/ubuntu/roofmap_project/datasets/bonn_dataset_shape/images/train/patch_5376_60160.png: ignoring corrupt image/label: image size (384, 3) <10 pixels



val: Scanning /home/ubuntu/roofmap_project/datasets/bonn_dataset_shape/labels/val.cache... 1190 images, 0 backgr


Plotting labels to /home/ubuntu/roofmap_project/runs/segment/run_5_shape/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to /home/ubuntu/roofmap_project/runs/segment/run_5_shape
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

     28/100      7.46G      1.484      2.463      1.413      1.046         88        320: 100%|██████████| 348/3
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R  


                   all       1190      21792        0.2      0.263      0.171      0.102      0.192       0.25       0.16     0.0797

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     29/100      7.46G      1.473       2.45      1.406      1.044        115        320: 100%|██████████| 348/3
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R  


                   all       1190      21792      0.272      0.263      0.186      0.114      0.266       0.25      0.173     0.0884

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     30/100      7.46G      1.458      2.419      1.386      1.034        444        320:  59%|█████▉    | 207/3

#### Next Steps + Potential Improvements

- Experimenting with smaller patch sizes to decrease the amount of patches with unlabelled houses
- Training using a custom model architecture
- Applying data augmentation to balance out the 
- Creating a pipeline that works for areas with sparse/non-grouped labels such as Nepal, Mozambique, Sri Lanka, and India. This was the initial goal but we realized data quality was subpar, so we pivoted to creating a reproducible pipeline in a data-rich area that could later be applied to other areas.
- Creating a pipeline that works for medium-high resolution satellite imagery
- Provide model checkpoints
- Evaluation and inference on nearby areas in Germany to expand available data
- Manually validating test set data
- Create roof shape dataset